# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

     |████████████████████████████████| 1.8 MB 2.1 MB/s 
     |████████████████████████████████| 181 kB 89.0 MB/s 
     |████████████████████████████████| 162 kB 70.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 158 kB 60.0 MB/s 
     |████████████████████████████████| 157 kB 85.3 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 86.0 MB/s 
     |████████████████████████████████| 157 kB 95.5 MB/s 
     |████████████████████████████████| 157 kB 92.8 MB/s 
     |████████████████████████████████| 157 kB 84.1 MB/s 
     |████████████████████████████████| 157 kB 90.8 MB/s 
     |████████████████████████████████| 156 kB 89.6 MB/s 


In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import wandb
from pathlib import Path
import sklearn.metrics
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ramanathan16","key":"d4035f55a52974a7c4d6bae8054bc215"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=a796cdbd0ec7b4ca70e6d5b4c65b3d5ee1303e6d681984598c62c86f8d9f8247
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-f22-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

100% 2.13G/2.13G [00:37<00:00, 120MB/s]
100% 2.13G/2.13G [00:37<00:00, 61.5MB/s]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
#from torch._C import int32
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        if partition == "train":
          self.mfcc_dir = self.data_path/ Path(partition+'-clean-100') /Path('mfcc') # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
          self.transcript_dir = self.data_path/ Path(partition+'-clean-100') /Path('transcript') # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data

        elif partition == "dev":
          self.mfcc_dir = self.data_path/ Path(partition +'-clean')/Path('mfcc')
          self.transcript_dir = self.data_path/ Path(partition+'-clean')/Path('transcript')


        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names = sorted(os.listdir(self.transcript_dir)) # TODO: List files in self.transcript_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []
        b= ['<sos>','<eos>']

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir/mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
            
            mean = np.mean(mfcc, axis=0)
            var = np.var(mfcc,axis=0)
            n_mfcc = (mfcc - mean)/var


        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir/transcript_names[i]) # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
            clean_transcript = np.delete(transcript, np.where(transcript == '<sos>'))
            clean_transcript = np.delete(clean_transcript, np.where(clean_transcript == '<eos>'))

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(n_mfcc)
            self.transcripts.append(clean_transcript)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs,axis =0)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts,axis =0)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc

        

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        map = {x: i for i,x in enumerate(self.phonemes)}

        for i in range(len(self.transcripts)):
            self.transcripts[i]= map[self.transcripts[i]]

        self.transcripts = np.asarray(self.transcripts, dtype=int)


        

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, pad_width=((self.context,self.context),(0,0)), mode="constant", constant_values= 0)
        self.transcripts= np.pad(self.transcripts, pad_width=((self.context, self.context)), mode="constant", constant_values= 0)

    def __len__(self):
      return self.length

    def __getitem__(self, ind):
      

      start= ind
      end = ind +(2*self.context+1)

        
      frames = self.mfccs[start:end] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
      # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
      frames = frames.flatten() # TODO: Flatten to get 1d data

      frames = torch.FloatTensor(frames) # Convert to tensors
      phoneme = torch.tensor(self.transcripts[ind+self.context])       

      return frames, phoneme




In [ ]:
test = AudioDataset(data_path='/content/data',context=1, offset=0, partition= "train")


KeyboardInterrupt: ignored

In [ ]:
print(test[0])

(tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 12.3430,
        -1.1226, -0.1066,  0.0684, -0.2572, -0.1075,  0.2360, -0.2117, -0.1686,
        -0.4367, -0.4411,  0.0822, -0.0688, -0.1083, -0.2555]), tensor(0))


In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
  def __init__(self, data_path, context, offset=0, partition= "test", limit=-1):

    self.context = context
    self.offset = offset
    self.data_path = data_path

    self.mfcc_dir = self.data_path/ Path(partition+'-clean') /Path('mfcc') # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
    
    mfcc_names = sorted(os.listdir(self.mfcc_dir)) 
   

    self.mfccs = []

    for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
        mfcc = np.load(self.mfcc_dir/mfcc_names[i])
        mean = np.mean(mfcc, axis=0)
        var = np.var(mfcc,axis=0)
        n_mfcc = (mfcc - mean)/var
        self.mfccs.append(n_mfcc)
  

    self.mfccs = np.concatenate(self.mfccs,axis =0)

    self.length = len(self.mfccs)

    self.mfccs = np.pad(self.mfccs, pad_width=((self.context,self.context),(0,0)), mode="constant", constant_values= 0)

  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    

    start= ind
    end = ind +(2*self.context+1)

    frames = self.mfccs[start:end]
    frames = frames.flatten()

    frames = torch.FloatTensor(frames)
    return frames





    

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs': 70,
    'batch_size' : 4096,
    'context' : 30,
    'learning_rate' : 0.001,
    'architecture' : 'medium-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
train_data = AudioDataset(data_path='/content/data',context=config['context'], offset=0, partition= "train") #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset(data_path='/content/data',context=config['context'], offset=0, partition= "dev") # TODO: Create a dataset object using the AudioDataset class for the validation data 



In [ ]:
a=val_data.transcripts
np.unique(a, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39]),
 array([396427,  33196,  55212,  88799,  27736,  21036,  47481,  22436,
         14144,  48215,  28445,  42820,  56102,  38939,  35969,  13627,
         26904,  63752,  81199,   9396,  46601,  73571,  43783,  85424,
         21730,  31452,   3752,  31371,  59326, 111876,  20220,  84166,
         11322,   5713,  20977,  24287,  37897,  11732,  59360,   1161]))

In [ ]:
test_data = AudioTestDataset(data_path='/content/data',context=config['context'], offset=0, partition= "test") # TODO: Create a dataset object using the AudioTestDataset class for the test data 

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  4096
Context:  30
Input size:  915
Output symbols:  42
Train dataset samples = 36191134, batches = 8836
Validation dataset samples = 1937496, batches = 474
Test dataset samples = 1943253, batches = 475


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([4096, 915]) torch.Size([4096])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case?
        output_size = 40 #Why?

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),
            torch.nn.Linear(2048,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.25),
            torch.nn.GELU(),

            torch.nn.Linear(2048,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.25),
            torch.nn.GELU(),


            torch.nn.Linear(2048,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.25),
            torch.nn.GELU(),

            torch.nn.Linear(2048,1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.25),
            torch.nn.GELU(),

            torch.nn.Linear(1024,1024),torch.nn.BatchNorm1d(1024),torch.nn.Dropout(0.25),torch.nn.GELU(),
            torch.nn.Linear(1024,1024),torch.nn.BatchNorm1d(1024),torch.nn.Dropout(0.25),torch.nn.GELU(),
            torch.nn.Linear(1024,1024),torch.nn.BatchNorm1d(1024),torch.nn.Dropout(0.25),torch.nn.GELU(),
            torch.nn.Linear(1024,output_size)
        )      

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
model = torch.load('./model_checkpoint_1.pth')

In [ ]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [915, 2048]  [4096, 2048]  1.875968M   1.87392M
1_model.Linear_1         [2048, 2048]  [4096, 2048]  4.196352M  4.194304M
2_model.BatchNorm1d_2          [2048]  [4096, 2048]     4.096k     2.048k
3_model.Dropout_3                   -  [4096, 2048]          -          -
4_model.GELU_4                      -  [4096, 2048]          -          -
5_model.Linear_5         [2048, 2048]  [4096, 2048]  4.196352M  4.194304M
6_model.BatchNorm1d_6          [2048]  [4096, 2048]     4.096k     2.048k
7_model.Dropout_7                   -  [4096, 2048]          -          -
8_model.GELU_8                      -  [4096, 2048]          -          -
9_model.Linear_9         [2048, 2048]  [4096, 2048]  4.196352M  4.194304M
10_model.BatchNorm1d_10        [2048]  [4096, 2048]     4.096k     2.048k
11_model.Dropout_11                 - 

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[915, 2048]","[4096, 2048]",1875968.0,1873920.0
1_model.Linear_1,"[2048, 2048]","[4096, 2048]",4196352.0,4194304.0
2_model.BatchNorm1d_2,[2048],"[4096, 2048]",4096.0,2048.0
3_model.Dropout_3,-,"[4096, 2048]",NaN,NaN
4_model.GELU_4,-,"[4096, 2048]",NaN,NaN
5_model.Linear_5,"[2048, 2048]","[4096, 2048]",4196352.0,4194304.0
6_model.BatchNorm1d_6,[2048],"[4096, 2048]",4096.0,2048.0
7_model.Dropout_7,-,"[4096, 2048]",NaN,NaN
8_model.GELU_8,-,"[4096, 2048]",NaN,NaN


In [ ]:
from torch.optim.lr_scheduler import StepLR,CosineAnnealingLR,ReduceLROnPlateau,ExponentialLR

criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
scheduler = CosineAnnealingLR(optimizer,T_max=5,eta_min=1e-5)
# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

765

In [ ]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in tqdm(enumerate(dataloader)):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
  
    train_loss /= len(dataloader)
    return train_loss

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="0f42a1910060e4a99487f7b1241e7d8a3d45182b") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "already 85.1 Exponential saved dc deepgelu 2048 with batch norm and dropout", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

Learning Rate,▁
train loss,▁
validation accuracy,▁
Learning Rate,0.003
train loss,0.81942
validation accuracy,78.42143


In [ ]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20220929_204817-3i2yuamo/files/model_arch.txt']

In [ ]:
optimizer.param_groups[0]['lr']

0.001

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
from torch.optim import adam
from tqdm import tqdm
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 85.5 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, train_loader)
    accuracy = eval(model, val_loader)
    learning_rate = optimizer.param_groups[0]['lr']



    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))
    print("\tLearning Rate: {:.8f}".format(learning_rate))

    scheduler.step(accuracy) #or on val_acc with max in scheudler

    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy, "Learning Rate": learning_rate})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint_checker.pth')
      
      ### Save checkpoint in wandb
      wandb.save('checkpoint_checker.pth')
 # scheduler.step(accuracy) #or on val_acc with max in scheudler

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/70


8836it [11:06, 13.26it/s]


	Train Loss: 0.3572
	Validation Accuracy: 85.06%
	Learning Rate: 0.00100000

Epoch 2/70


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
8836it [11:06, 13.25it/s]


	Train Loss: 0.3545
	Validation Accuracy: 85.07%
	Learning Rate: 0.00001031

Epoch 3/70


...

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
8836it [11:06, 13.26it/s]


	Train Loss: 0.3527
	Validation Accuracy: 85.08%
	Learning Rate: 0.00001070

Epoch 30/70


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
8836it [11:06, 13.26it/s]


	Train Loss: 0.3527
	Validation Accuracy: 85.07%
	Learning Rate: 0.00001056

Epoch 31/70


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
8705it [10:52, 13.33it/s]

In [ ]:
torch.save(model,'./model_checkpoint_2.pth')

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.no_grad():

      test_predictions= [ ]
      

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)
        

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = output.argmax(dim=1)
          
          test_predictions.append(predicted_phonemes)

  prediction = torch.cat(test_predictions,0)

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          
  return prediction

In [ ]:
predictions = test(model, test_loader)

100%|██████████| 475/475 [00:13<00:00, 36.31it/s]


In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"

100% 18.6M/18.6M [00:05<00:00, 3.44MB/s]
Successfully submitted to Frame-Level Speech Recognition